In [52]:
import pandas as pd
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder

In [9]:
# Ruta al archivo CSV
csv_file_path = 'cleaned_sucesos_updated.csv'

# Cargar los datos del archivo CSV en la variable 'data'
data = pd.read_csv(csv_file_path)

In [10]:
# Función para convertir la hora en un número entero
def convert_time_to_hour(X):
    return pd.to_datetime(X['time']).dt.hour.values.reshape(-1, 1)

In [13]:
#CON HORA Y SOLO LOCATION DATA

# Crear un transformador para la columna 'time'
time_transformer = FunctionTransformer(convert_time_to_hour, validate=False)

categorical_features = ['commune']

# Actualizar el preprocesador para incluir la transformación de 'time'
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('time', time_transformer, ['time'])],
    remainder='passthrough')

# Aplicar preprocesamiento a los datos (incluyendo 'time')
X = data.drop(['datetime', 'severity', 'injury_type', 'rest_days' ], axis=1)  # Incluir 'time' excluir data de accidente
y = data['severity', 'rest_days']
X_processed = preprocessor.fit_transform(X)

print(X)
print(y)

# # Dividir el conjunto de datos en entrenamiento y prueba
# X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

# # Entrenar el modelo de Bosque Aleatorio
# model = RandomForestClassifier(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# # Evaluar el modelo actualizado
# predictions = model.predict(X_test)
# classification_report_results_updated = classification_report(y_test, predictions)

# print(classification_report_results_updated)

KeyError: ('severity', 'rest_days')


La severidad predicha es: 1


In [6]:
with open('nostradamus/preprocessor.pkl', 'wb') as file:
    pickle.dump(preprocessor, file)
    
with open('nostradamus/model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("guardado")

guardado


In [7]:
import sklearn
print(sklearn.__version__)

1.4.0


In [ ]:
### Limpiad los datos

In [3]:
import pandas as pd
import json

def obtener_valor_seguro(diccionario, ruta, valor_por_defecto=''):
    try:
        for clave in ruta:
            diccionario = diccionario[clave]
        return diccionario
    except (KeyError, IndexError, TypeError):
        return valor_por_defecto

def cargar_y_convertir_json_a_csv(ruta_json, ruta_csv):
    # Cargar datos del archivo JSON
    with open(ruta_json, 'r') as archivo:
        datos = json.load(archivo)

    # Preparar una lista para los datos del CSV
    datos_csv = []

    for suceso in datos:
        arcgis = suceso['arcgis']

        # Extracción de datos y almacenamiento en un diccionario
        fila = {
            "timestamp": suceso['atdn_atd_fecha_ocurrencia'],
            "date": suceso['atdn_atd_fecha_ocurrencia'].split()[0],
            "time": suceso['atdn_atd_fecha_ocurrencia'].split()[1],
            "gender": suceso['pcnt_genero'],
            "age": suceso['pcnt_edad'],
            
            # Para origin (employeeCoordinate)
            "origin_address": obtener_valor_seguro(arcgis, ['employeeCoordinate', 'candidates', 0, 'address']),
            "origin_score": obtener_valor_seguro(arcgis, ['employeeCoordinate', 'candidates', 0, 'score']),
            "origin_longitude": obtener_valor_seguro(arcgis, ['employeeCoordinate', 'candidates', 0, 'location', 'x']),
            "origin_latitude": obtener_valor_seguro(arcgis, ['employeeCoordinate', 'candidates', 0, 'location', 'y']),
            "origin_extent_xmin": obtener_valor_seguro(arcgis, ['employeeCoordinate', 'candidates', 0, 'extent', 'xmin']),
            "origin_extent_ymin": obtener_valor_seguro(arcgis, ['employeeCoordinate', 'candidates', 0, 'extent', 'ymin']),
            "origin_extent_xmax": obtener_valor_seguro(arcgis, ['employeeCoordinate', 'candidates', 0, 'extent', 'xmax']),
            "origin_extent_ymax": obtener_valor_seguro(arcgis, ['employeeCoordinate', 'candidates', 0, 'extent', 'ymax']),

            # Para accident (accidentCoordinate)
            "accident_address": obtener_valor_seguro(arcgis, ['accidentCoordinate', 'candidates', 0, 'address']),
            "accident_score": obtener_valor_seguro(arcgis, ['accidentCoordinate', 'candidates', 0, 'score']),
            "accident_longitude": obtener_valor_seguro(arcgis, ['accidentCoordinate', 'candidates', 0, 'location', 'x']),
            "accident_latitude": obtener_valor_seguro(arcgis, ['accidentCoordinate', 'candidates', 0, 'location', 'y']),
            "accident_extent_xmin": obtener_valor_seguro(arcgis, ['accidentCoordinate', 'candidates', 0, 'extent', 'xmin']),
            "accident_extent_ymin": obtener_valor_seguro(arcgis, ['accidentCoordinate', 'candidates', 0, 'extent', 'ymin']),
            "accident_extent_xmax": obtener_valor_seguro(arcgis, ['accidentCoordinate', 'candidates', 0, 'extent', 'xmax']),
            "accident_extent_ymax": obtener_valor_seguro(arcgis, ['accidentCoordinate', 'candidates', 0, 'extent', 'ymax']),

            # Para destination (companyCoordinate)
            "destination_address": obtener_valor_seguro(arcgis, ['companyCoordinate', 'candidates', 0, 'address']),
            "destination_score": obtener_valor_seguro(arcgis, ['companyCoordinate', 'candidates', 0, 'score']),
            "destination_longitude": obtener_valor_seguro(arcgis, ['companyCoordinate', 'candidates', 0, 'location', 'x']),
            "destination_latitude": obtener_valor_seguro(arcgis, ['companyCoordinate', 'candidates', 0, 'location', 'y']),
            "destination_extent_xmin": obtener_valor_seguro(arcgis, ['companyCoordinate', 'candidates', 0, 'extent', 'xmin']),
            "destination_extent_ymin": obtener_valor_seguro(arcgis, ['companyCoordinate', 'candidates', 0, 'extent', 'ymin']),
            "destination_extent_xmax": obtener_valor_seguro(arcgis, ['companyCoordinate', 'candidates', 0, 'extent', 'xmax']),
            "destination_extent_ymax": obtener_valor_seguro(arcgis, ['companyCoordinate', 'candidates', 0, 'extent', 'ymax']),

            "origin_comuna": suceso['bde_comuna_acc'],
            "accident_comuna": suceso['bde_comuna_adherente'],
            "destination_comuna": suceso['bde_comuna_trabajador'],
            "vehicle_type": suceso['vehiculo_paciente'],
            "injury_type": suceso['caso_nom_diag_1_may_est'],
            "ocupation": suceso['caso_profesion_ocupacion'],
            "severity": suceso['grave'],
            "restdays": suceso['rpss_dias_reposo']
        }
        datos_csv.append(fila)

    # Convertir la lista de diccionarios en un DataFrame de pandas
    df = pd.DataFrame(datos_csv)

    # Guardar el DataFrame en un archivo CSV
    df.to_csv(ruta_csv, index=False)

# Llamar a la función con las rutas del archivo JSON y del archivo CSV deseado
cargar_y_convertir_json_a_csv('datasets/sucesos.json', 'datasets/sucesos.csv')

In [4]:
# Función para convertir la hora en un número entero
def convert_time_to_hour(X):
    return pd.to_datetime(X['time']).dt.hour.values.reshape(-1, 1)

def convert_date_to_day_of_year(X):
    date_series = pd.to_datetime(X['date'])
    day_of_year = date_series.dt.dayofyear
    return day_of_year.values.reshape(-1, 1)

# Crear un transformador para la columna 'time'
time_transformer = FunctionTransformer(convert_time_to_hour, validate=False)

# Crear un transformador para la columna 'date'
date_transformer = FunctionTransformer(convert_date_to_day_of_year, validate=False)

In [56]:
# Ruta al archivo CSV
csv_file_path = 'datasets/sucesos.csv'

columns = [
    "time",
    "gender",
    "age",
    "origin_longitude",
    "origin_latitude",
    "accident_longitude",
    "accident_latitude",
    "destination_longitude",
    "destination_latitude",
    "vehicle_type",
    "restdays"
]

# Cargar los datos del archivo CSV en la variable 'data'
data = pd.read_csv(csv_file_path, usecols=columns)

In [57]:
nan_count = data.isna().sum()

# Mostrar el recuento de NaN para cada columna
print("Recuento de valores NaN por columna:\n", nan_count)

# Contar el total de valores NaN en todo el DataFrame
total_nan_count = data.isna().sum().sum()
print("\nTotal de valores NaN en el DataFrame:", total_nan_count)

Recuento de valores NaN por columna:
 time                         0
gender                       0
age                          1
origin_longitude         67854
origin_latitude          67854
accident_longitude       67848
accident_latitude        67848
destination_longitude    67849
destination_latitude     67849
vehicle_type                 0
restdays                     0
dtype: int64

Total de valores NaN en el DataFrame: 407103


In [58]:
data = data.dropna()

# Verificar el nuevo tamaño del DataFrame
print("Nuevo tamaño del DataFrame después de eliminar NaNs:", data.shape)
nan_count = data.isna().sum()

# Mostrar el recuento de NaN para cada columna
print("Recuento de valores NaN por columna:\n", nan_count)

# Contar el total de valores NaN en todo el DataFrame
total_nan_count = data.isna().sum().sum()
print("\nTotal de valores NaN en el DataFrame:", total_nan_count)

Nuevo tamaño del DataFrame después de eliminar NaNs: (63293, 11)
Recuento de valores NaN por columna:
 time                     0
gender                   0
age                      0
origin_longitude         0
origin_latitude          0
accident_longitude       0
accident_latitude        0
destination_longitude    0
destination_latitude     0
vehicle_type             0
restdays                 0
dtype: int64

Total de valores NaN en el DataFrame: 0


In [59]:
unique_vehicle_types = data['vehicle_type'].unique()

# Convertimos el array de valores únicos a un DataFrame para guardarlo fácilmente
unique_vehicle_types_df = pd.DataFrame(unique_vehicle_types, columns=['Unique Vehicle Types'])

# Guardamos los valores únicos en un archivo CSV
unique_vehicle_types_df.to_csv('unique_vehicle_types.csv', index=False)

In [43]:
categorical_features = ["gender", "vehicle_type"]

# ColumnTransformer para preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('time', time_transformer, ['time'])
    ], 
    remainder='passthrough'
)

columns_inf = ["restdays"]
X = data.drop(columns_inf, axis=1)
y = data[columns_inf]
X_processed = preprocessor.fit_transform(X)

/var/folders/v3/kqxwyt1s0dz1b8kyz1knsvn00000gn/T/ipykernel_83198/1611795929.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X['time']).dt.hour.values.reshape(-1, 1)


In [44]:
print("XDtypes")
print(X.dtypes)
print("YDtypes")
print(y.dtypes)
print("Características de entrada (X):")
print(X.head())
print("\nSalidas (y):")
print(y.head())

XDtypes
time                      object
gender                    object
age                      float64
origin_longitude         float64
origin_latitude          float64
accident_longitude       float64
accident_latitude        float64
destination_longitude    float64
destination_latitude     float64
vehicle_type              object
dtype: object
YDtypes
restdays    int64
dtype: object
Características de entrada (X):
       time gender   age  origin_longitude  origin_latitude  \
0  18:30:00      M  69.0        -71.579547       -35.857849   
1  17:20:00      F  34.0        -70.589200       -33.550360   
2  16:30:00      F  21.0        -70.670120       -33.381670   
3  18:30:00      M  38.0        -70.528270       -33.613070   
4  18:00:00      F  44.0        -70.664820       -33.389810   

   accident_longitude  accident_latitude  destination_longitude  \
0          -71.593360          -35.85227             -71.595413   
1          -70.598570          -33.41845             -70.598880

In [45]:
# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

# Entrenar el modelo de Bosque Aleatorio
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo actualizado
predictions = model.predict(X_test)
classification_report_results_updated = classification_report(y_test, predictions)

print(classification_report_results_updated)

/Users/aecheverry/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       0.28      0.77      0.41      4692
           1       0.11      0.05      0.07      1227
           2       0.14      0.13      0.14      2122
           3       0.08      0.06      0.07      1748
           4       0.04      0.01      0.02       902
           5       0.07      0.02      0.03       875
           6       0.03      0.01      0.01       431
           7       0.05      0.01      0.02       525
           8       0.04      0.01      0.02       564
           9       0.00      0.00      0.00       366
          10       0.06      0.01      0.02       326
          11       0.00      0.00      0.00       253
          12       0.00      0.00      0.00       259
          13       0.00      0.00      0.00       167
          14       0.00      0.00      0.00       197
          15       0.00      0.00      0.00       229
          16       0.04      0.01      0.01       130
          17       0.00    

/Users/aecheverry/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aecheverry/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aecheverry/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Asumiendo que 'X_processed' es su conjunto de características y 'y' es la variable objetivo (días de descanso)
# Asegúrese de que 'y' sea un vector con los valores a predecir (días de descanso)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_processed, y['restdays'], test_size=0.3, random_state=42)

# Entrenar el modelo de regresión lineal
model_restdays = LinearRegression()
model_restdays.fit(X_train, y_train)

# Evaluar el modelo
predictions = model_restdays.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

# Calcular la raíz del error cuadrático medio (opcional, para una interpretación más intuitiva)
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Mean Squared Error: 4034.6138875882666
Root Mean Squared Error: 63.51861056090779


In [49]:
def predecir_severidad(model, preprocessor, new_data):
    """
    Predecir la severidad de un nuevo accidente.

    :param model: El modelo de clasificación entrenado.
    :param preprocessor: El transformador de preprocesamiento utilizado en los datos de entrenamiento.
    :param new_data: Diccionario con los detalles del nuevo accidente.
    :return: Predicción de la severidad del accidente.
    """
    # Convertir new_data en un DataFrame
    new_data_df = pd.DataFrame([new_data])

    # Aplicar el mismo preprocesamiento utilizado en el modelo
    new_data_processed = preprocessor.transform(new_data_df)

    # Predecir la severidad
    severity_prediction = model.predict(new_data_processed)
    print()
    return severity_prediction[0]

# Ejemplo de uso
new_accident = {
    "time": "18:30:00",
    "gender": "M",
    "age": 65,
    "origin_longitude": -71.5933600308,
    "origin_latitude": -35.8522700127,
    "accident_longitude": -71.5933600308,
    "accident_latitude": -35.8522700127,
    "destination_longitude": -71.5933600308,
    "destination_latitude": -35.8522700127,
    "vehicle_type": "Bicicleta/Scooter"
}

# Reemplazar 'model' y 'preprocessor'
severity = predecir_severidad(model_restdays, preprocessor, new_accident)
print(f"La severidad predicha es: {severity}")


La severidad predicha es: 43.639652252197266


In [51]:
# with open('preprocessor.pkl', 'wb') as file:
#     pickle.dump(preprocessor, file)
    
with open('model_restdays.pkl', 'wb') as file:
    pickle.dump(model_restdays, file)

print("guardado")

guardado


In [35]:
# Suponiendo que 'data' es su DataFrame y contiene una columna 'timestamp'
data['date'] = pd.to_datetime(data['timestamp']).dt.date
accidents_per_day = data.groupby('date').size()

# Calculando la probabilidad diaria
# Aquí, 750000 es el número promedio de personas en movimiento cada hora
daily_probability = accidents_per_day / 750000

In [37]:
# Convertir fechas a día del año
data['day_of_year'] = pd.to_datetime(data['date']).dt.dayofyear

# Agrupar por día del año y calcular la media de accidentes
average_accidents_per_day = data.groupby('day_of_year').size()

# Crear el diccionario de probabilidades
probability_dict = {day: accidents / 750000 for day, accidents in average_accidents_per_day.items()}

In [38]:
print(probability_dict)

{1: 5.466666666666667e-05, 2: 0.00042133333333333335, 3: 0.00040666666666666667, 4: 0.00040533333333333334, 5: 0.0003786666666666667, 6: 0.0004133333333333333, 7: 0.00043466666666666664, 8: 0.0004946666666666667, 9: 0.0005293333333333334, 10: 0.00046, 11: 0.000488, 12: 0.00034, 13: 0.00046133333333333334, 14: 0.0004893333333333334, 15: 0.0005493333333333333, 16: 0.00046666666666666666, 17: 0.000448, 18: 0.00044, 19: 0.00038666666666666667, 20: 0.000464, 21: 0.0005186666666666666, 22: 0.00045066666666666665, 23: 0.0004986666666666667, 24: 0.0003773333333333333, 25: 0.00039466666666666665, 26: 0.0003986666666666667, 27: 0.00048666666666666666, 28: 0.0005626666666666667, 29: 0.000528, 30: 0.00048266666666666667, 31: 0.0004186666666666667, 32: 0.000408, 33: 0.000364, 34: 0.0004186666666666667, 35: 0.00043733333333333336, 36: 0.00042533333333333334, 37: 0.0004946666666666667, 38: 0.0003786666666666667, 39: 0.000464, 40: 0.000392, 41: 0.0004493333333333333, 42: 0.000412, 43: 0.000416, 44: 0.

In [39]:
import pickle

with open('probability.pkl', 'wb') as file:
    pickle.dump(probability_dict, file)
